# Visualización de la Distribución de Estudiantes por Sexo, Rama Educacional y Región

## 1. Importación de Librerías

En esta primera sección, importamos las librerías necesarias para manipular los datos y realizar las visualizaciones. Se utiliza **pandas** para la manipulación de datos y **plotly.express** para la creación de gráficos interactivos.

In [111]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


## 2. Carga de Datos

A continuación, cargamos los datos procesados del archivo CSV que contiene la información de los estudiantes inscritos en el proceso de admisión PAES 2025. Estos datos ya han sido previamente limpiados y estructurados en un archivo CSV.

In [112]:
# Función para cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

## 3. Mapeo de Variables

Para mejorar la legibilidad de los datos, se realiza el mapeo de valores numéricos a categorías. En esta sección, asignamos nombres descriptivos a las ramas educacionales y las regiones, así como a los valores correspondientes al sexo (1 = Hombre, 2 = Mujer).

In [113]:
data['sexo'] = data['sexo'].map({1: 'Hombre', 2: 'Mujer'})

### Mapeo de Ramas Educacionales:


In [114]:
# Diccionario para mapear ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

### Mapeo de Regiones:


In [115]:
# Diccionario para mapear regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

## 4. Cálculo de la Distribución

Calculamos la distribución de los estudiantes agrupados por sexo, rama educacional y región. También se calcula el porcentaje de cada grupo con respecto al total de estudiantes de cada sexo.

In [116]:
# Función para calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

# Calculamos la distribución
distribution = calcular_distribucion(data)

## 5. Visualización con Treemap

Utilizamos un gráfico treemap para visualizar la distribución de los estudiantes, clasificándolos por sexo, rama educacional y región. Cada cuadro en el gráfico representa un subconjunto de estudiantes, y el tamaño del cuadro está relacionado con el porcentaje de ese grupo dentro de su categoría.

Los colores se asignan según el sexo, con azul para los hombres y rosa para las mujeres, facilitando la identificación visual.

In [118]:
# Función para crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(
        distribution, 
        path=['general_label', 'sexo', 'label_with_percentage'], 
        values='percentage', 
        color='percentage',  
        color_continuous_scale='Viridis',  
        labels={'percentage': 'Distribución de estudiantes en %'},
        hover_data={'count': True}
    )
    
    # Personalización del diseño
    fig.update_layout(
        height=1000,  
        margin=dict(t=200, b=150),
        coloraxis_colorbar=dict(
            title=dict(
                text="<b>Distribución de<br>estudiantes en %</b>",  
                font=dict(size=18, family="Econ Sans Cnd")
            ),
            orientation='h',  
            x=0.79,  
            y=1.02,  
            thickness=17,
            len=0.42,
            tickvals=[0, 5, 10, 15, 20, 25, 30, 35, 40],
            ticktext=['0', '5', '10', '15', '20', '25', '30', '35', '40'],
            tickfont=dict(size=16),
            ticks="outside",
            tickwidth=1,
            ticklen=10
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,
                text="<b>Análisis de Admisiones PAES 2025</b>",
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Anotaciones inferiores
            dict(
                x=0, y=-0.05,
                text='<b>Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile</b>',
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.05,
                text="<b>Nota: Datos correspondientes al proceso de admisión 2025</b>",
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,
                line=dict(color="red", width=1),
                xref="paper", yref="paper"
            ),
            # Rectángulo decorativo
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustes adicionales al treemap
    fig.update_traces(
        marker=dict(line=dict(width=2)),
        textinfo='label+text+value',
        textfont_size=18,  
        texttemplate='%{label}<br>%{value}%',
        tiling=dict(pad=1, squarifyratio=1.0)
    )
    
    fig.show()

# Generamos el treemap
crear_treemap(distribution)